In [1]:
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import keras.callbacks
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 100
num_classes = 10
epochs = 100#
input_shape = (784,)

img_rows, img_cols = 28, 28

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


60000 train samples
10000 test samples


In [4]:
class AccuracyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0:
            print('Epoch num:', epoch, '---', logs)

In [5]:
learning_rate = 0.5
epochs = 3 #epochs = 100

linear_model = Sequential()
linear_model.add(Dense(num_classes, activation='softmax', input_shape=(784,), use_bias=True, bias_initializer='zeros'))

sgd = keras.optimizers.SGD(lr=learning_rate)
linear_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

linear_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_data=(x_test, y_test),
          callbacks=[AccuracyCallback()])

score = linear_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch num: 0 --- {'val_loss': 0.3132649681158364, 'val_accuracy': 0.9136999845504761, 'loss': 0.39378708499173326, 'accuracy': 0.8874}
10000/10000 [==============================] - 1s 56us/step
Test loss: 0.28003800650835037
Test accuracy: 0.921500027179718


In [6]:
hidden_neurons =1500
iterations = 20000
learning_rate = 10e-3
b1 = 0.9
b2 = 0.999
epsilon = 10e-8
epochs = 3 #epochs = int(iterations/(x_train.shape[0]/batch_size))



mlp_model = Sequential()

weight_init = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)
bias_init = keras.initializers.Constant(value=-0.1)
adam_optim = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=b1, beta_2=b2, amsgrad=False)

mlp_model.add(Dense(hidden_neurons, activation='relu', 
                       input_shape=(784,), use_bias=True, bias_initializer=bias_init, kernel_initializer=weight_init))
mlp_model.add(Dense(hidden_neurons, activation='relu', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))
mlp_model.add(Dense(hidden_neurons, activation='relu', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))
mlp_model.add(Dense(num_classes, activation='softmax', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))

mlp_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=adam_optim,
              metrics=['accuracy'])

mlp_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = mlp_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 122s 2ms/step - loss: 2.3020 - accuracy: 0.1109 - val_loss: 2.3024 - val_accuracy: 0.1028

In [7]:
hidden_neurons =1500
iterations = 20000
learning_rate = 10e-3
dropout = 0.5
b1 = 0.9
b2 = 0.999
epsilon = 10e-8
epochs = 3 # epochs = int(iterations/(x_train.shape[0]/batch_size))

dropout_model = Sequential()

weight_init = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)
bias_init = keras.initializers.Constant(value=-0.1)
adam_optim = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=b1, beta_2=b2, amsgrad=False)

dropout_model.add(Dense(hidden_neurons, activation='relu', 
                       input_shape=(784,), use_bias=True, bias_initializer=bias_init, kernel_initializer=weight_init))
dropout_model.add(Dropout(dropout))
dropout_model.add(Dense(hidden_neurons, activation='relu', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))
dropout_model.add(Dropout(dropout))
dropout_model.add(Dense(hidden_neurons, activation='relu', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))
dropout_model.add(Dropout(dropout))
dropout_model.add(Dense(num_classes, activation='softmax', use_bias=True,
                       bias_initializer=bias_init, kernel_initializer=weight_init))

dropout_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=adam_optim,
              metrics=['accuracy'])

dropout_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = dropout_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 136s 2ms/step - loss: 2.3020 - accuracy: 0.1101 - val_loss: 2.3020 - val_accuracy: 0.1028

In [8]:
iterations = 20000
learning_rate = 10e-3
dropout = 0.5
b1 = 0.9
b2 = 0.999
epsilon = 10e-8
epochs = 3 # epochs = int(iterations/(x_train.shape[0]/batch_size))
cnn_kernel = (5,5)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

cnn_model = Sequential()

weight_init = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01, seed=None)
bias_init = keras.initializers.Constant(value=-0.1)
adam_optim = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=b1, beta_2=b2, amsgrad=False)

cnn_model.add(Conv2D(32, kernel_size=cnn_kernel,
                 activation='relu',
                 input_shape=input_shape, padding='same', strides = 1))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))
cnn_model.add(Conv2D(64, kernel_size=cnn_kernel,
                 activation='relu', padding='same', strides = 1))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))
cnn_model.add(Flatten())
cnn_model.add(Dense(output_dim = num_classes, activation='softmax', kernel_initializer=weight_init))

cnn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=adam_optim,
              metrics=['accuracy'])

cnn_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = cnn_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 233s 4ms/step - loss: 0.1384 - accuracy: 0.9579 - val_loss: 0.0526 - val_accuracy: 0.9832

/home/david/VirtualEnvs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", kernel_initializer=<keras.ini..., units=10)`
